# Retail Forecasting

In [1]:
import os
import numpy as np
import pandas as pd

### Analyze file size

In [2]:
# Define the path to data directory and file
data_path = 'data/'
filename = data_path+'retail.csv'

In [3]:
# Get the size of the data file
file_bytes = os.path.getsize(filename) 

print(f'File size is {file_bytes} bytes.')
print(f'File size is {np.round((file_bytes/1024),2)} KB.')
print(f'File size is {np.round((file_bytes/1024**2),2)} MB.')

File size is 51509 bytes.
File size is 50.3 KB.
File size is 0.05 MB.


### Analyze global data characteristics

In [4]:
# Load the data from the data file
df = pd.read_csv(filename)

In [5]:
# Get number of rows and columns in the data
df.shape

(1218, 12)

In [6]:
# Explore top five rows and bottom five rows of data
df

,Product,date,Sales,Price Discount (%),In-Store Promo,Catalogue Promo,Store End Promo,Google_Mobility,Covid_Flag,V_DAY,EASTER,CHRISTMAS
0,SKU1,2/5/2017,27750,0%,0,0,0,0.00,0,0,0,0
1,SKU1,2/12/2017,29023,0%,1,0,1,0.00,0,1,0,0
2,SKU1,2/19/2017,45630,17%,0,0,0,0.00,0,0,0,0
3,SKU1,2/26/2017,26789,0%,1,0,1,0.00,0,0,0,0
4,SKU1,3/5/2017,41999,17%,0,0,0,0.00,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1213,SKU6,10/18/2020,96619,54%,0,1,0,-7.56,1,0,0,0
1214,SKU6,10/25/2020,115798,52%,0,1,0,-8.39,1,0,0,0
1215,SKU6,11/1/2020,152186,54%,1,0,1,-7.43,1,0,0,0
1216,SKU6,11/8/2020,26445,44%,1,0,1,-5.95,1,0,0,0


In [7]:
# Get names, data types and non-null value counts of columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Product             1218 non-null   object 
 1   date                1218 non-null   object 
 2   Sales               1218 non-null   int64  
 3   Price Discount (%)  1218 non-null   object 
 4   In-Store Promo      1218 non-null   int64  
 5   Catalogue Promo     1218 non-null   int64  
 6   Store End Promo     1218 non-null   int64  
 7   Google_Mobility     1218 non-null   float64
 8   Covid_Flag          1218 non-null   int64  
 9   V_DAY               1218 non-null   int64  
 10  EASTER              1218 non-null   int64  
 11  CHRISTMAS           1218 non-null   int64  
dtypes: float64(1), int64(8), object(3)
memory usage: 114.3+ KB


### Analyze data granularity

#### Analyze `Product` (as a data granular) feature

In [8]:
# Strip leading and trailing whitespaces from the 'Product' feature
df['Product'] = df['Product'].str.strip()
# Replace empty strings with NaN in the 'Product' feature
df['Product'] = df['Product'].replace('', np.nan)
# Count the number of NaN values in the 'Product' feature
df['Product'].isnull().sum()

0

In [9]:
# Get unique 'Product' names 
products = df['Product'].sort_values().unique()

products

array(['SKU1', 'SKU2', 'SKU3', 'SKU4', 'SKU5', 'SKU6'], dtype=object)

#### Split the data into granular time-series

In [10]:
# Split the data into granular time-series based on unique product names
df_product_dict = {product: df[df['Product']==product] for product in products}
# Display time-series of one product
df_product_dict['SKU1']

,Product,date,Sales,Price Discount (%),In-Store Promo,Catalogue Promo,Store End Promo,Google_Mobility,Covid_Flag,V_DAY,EASTER,CHRISTMAS
0,SKU1,2/5/2017,27750,0%,0,0,0,0.00,0,0,0,0
1,SKU1,2/12/2017,29023,0%,1,0,1,0.00,0,1,0,0
2,SKU1,2/19/2017,45630,17%,0,0,0,0.00,0,0,0,0
3,SKU1,2/26/2017,26789,0%,1,0,1,0.00,0,0,0,0
4,SKU1,3/5/2017,41999,17%,0,0,0,0.00,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
199,SKU1,11/29/2020,0,16%,1,0,1,-5.30,1,0,0,0
200,SKU1,12/6/2020,0,15%,1,0,1,-4.38,1,0,0,0
201,SKU1,12/13/2020,0,14%,1,0,1,-4.74,1,0,0,0
202,SKU1,12/20/2020,0,15%,1,0,1,-10.27,1,0,0,1


### Analyze `Price Discount (%)` feature

In [11]:
for product, df_product in df_product_dict.items():
    df_product = df_product.copy()
    # Strip leading and trailing whitespaces and '%' sign from the 'Price Discount (%)' feature
    df_product['Price Discount (%)'] = df_product['Price Discount (%)'].str.strip('%').str.strip()
    # Convert the 'Price Discount (%)' feature to numeric format, handling errors by coercing invalid values into NaN
    df_product['Price Discount (%)'] = pd.to_numeric(df_product['Price Discount (%)'], errors='coerce')
    # Update df_product
    df_product_dict[product] = df_product

In [12]:
# Display time-series of one product
df_product_dict['SKU1']

,Product,date,Sales,Price Discount (%),In-Store Promo,Catalogue Promo,Store End Promo,Google_Mobility,Covid_Flag,V_DAY,EASTER,CHRISTMAS
0,SKU1,2/5/2017,27750,0,0,0,0,0.00,0,0,0,0
1,SKU1,2/12/2017,29023,0,1,0,1,0.00,0,1,0,0
2,SKU1,2/19/2017,45630,17,0,0,0,0.00,0,0,0,0
3,SKU1,2/26/2017,26789,0,1,0,1,0.00,0,0,0,0
4,SKU1,3/5/2017,41999,17,0,0,0,0.00,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
199,SKU1,11/29/2020,0,16,1,0,1,-5.30,1,0,0,0
200,SKU1,12/6/2020,0,15,1,0,1,-4.38,1,0,0,0
201,SKU1,12/13/2020,0,14,1,0,1,-4.74,1,0,0,0
202,SKU1,12/20/2020,0,15,1,0,1,-10.27,1,0,0,1


In [13]:
# For each product time-series, count the number of missing (null) values in the 'Price Discount (%)' feature left after type conversion
{product: df_product['Price Discount (%)'].isnull().sum() for product, df_product in df_product_dict.items()}

{'SKU1': 0, 'SKU2': 0, 'SKU3': 0, 'SKU4': 0, 'SKU5': 0, 'SKU6': 0}

In [14]:
for product, df_product in df_product_dict.items():
    df_product = df_product.copy()
    # Convert percentage into decimal format
    df_product['Price Discount (%)'] = df_product['Price Discount (%)']/100.0
    # Rename feature into 'Price Discount'
    df_product = df_product.rename(columns={'Price Discount (%)':'Price Discount'})
    # Update df_product
    df_product_dict[product] = df_product

In [15]:
# Display time-series of one product
df_product_dict['SKU1']

,Product,date,Sales,Price Discount,In-Store Promo,Catalogue Promo,Store End Promo,Google_Mobility,Covid_Flag,V_DAY,EASTER,CHRISTMAS
0,SKU1,2/5/2017,27750,0.00,0,0,0,0.00,0,0,0,0
1,SKU1,2/12/2017,29023,0.00,1,0,1,0.00,0,1,0,0
2,SKU1,2/19/2017,45630,0.17,0,0,0,0.00,0,0,0,0
3,SKU1,2/26/2017,26789,0.00,1,0,1,0.00,0,0,0,0
4,SKU1,3/5/2017,41999,0.17,0,0,0,0.00,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
199,SKU1,11/29/2020,0,0.16,1,0,1,-5.30,1,0,0,0
200,SKU1,12/6/2020,0,0.15,1,0,1,-4.38,1,0,0,0
201,SKU1,12/13/2020,0,0.14,1,0,1,-4.74,1,0,0,0
202,SKU1,12/20/2020,0,0.15,1,0,1,-10.27,1,0,0,1


In [16]:
# Get names, data types and non-null value counts of columns
df_product_dict['SKU1'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 204 entries, 0 to 203
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Product          204 non-null    object 
 1   date             204 non-null    object 
 2   Sales            204 non-null    int64  
 3   Price Discount   204 non-null    float64
 4   In-Store Promo   204 non-null    int64  
 5   Catalogue Promo  204 non-null    int64  
 6   Store End Promo  204 non-null    int64  
 7   Google_Mobility  204 non-null    float64
 8   Covid_Flag       204 non-null    int64  
 9   V_DAY            204 non-null    int64  
 10  EASTER           204 non-null    int64  
 11  CHRISTMAS        204 non-null    int64  
dtypes: float64(2), int64(8), object(2)
memory usage: 20.7+ KB


### Analyze `date` feature

In [17]:
for product, df_product in df_product_dict.items():
    df_product = df_product.copy()
    # Convert the 'date' feature to datetime format, handling errors by coercing invalid values into NaT
    df_product['date'] = pd.to_datetime(df_product['date'], errors='coerce')
    # Sort the data by 'date' feature values in ascending order
    df_product = df_product.sort_values(['date'])
    # Update df_product
    df_product_dict[product] = df_product

In [18]:
# Display time-series of one product
df_product_dict['SKU1']

,Product,date,Sales,Price Discount,In-Store Promo,Catalogue Promo,Store End Promo,Google_Mobility,Covid_Flag,V_DAY,EASTER,CHRISTMAS
0,SKU1,2017-02-05,27750,0.00,0,0,0,0.00,0,0,0,0
1,SKU1,2017-02-12,29023,0.00,1,0,1,0.00,0,1,0,0
2,SKU1,2017-02-19,45630,0.17,0,0,0,0.00,0,0,0,0
3,SKU1,2017-02-26,26789,0.00,1,0,1,0.00,0,0,0,0
4,SKU1,2017-03-05,41999,0.17,0,0,0,0.00,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
199,SKU1,2020-11-29,0,0.16,1,0,1,-5.30,1,0,0,0
200,SKU1,2020-12-06,0,0.15,1,0,1,-4.38,1,0,0,0
201,SKU1,2020-12-13,0,0.14,1,0,1,-4.74,1,0,0,0
202,SKU1,2020-12-20,0,0.15,1,0,1,-10.27,1,0,0,1


In [19]:
# For each product time-series, count the number of missing (null) values in the 'date' feature left after type conversion
{product: df_product['date'].isnull().sum() for product, df_product in df_product_dict.items()}

{'SKU1': 0, 'SKU2': 0, 'SKU3': 0, 'SKU4': 0, 'SKU5': 0, 'SKU6': 0}

In [20]:
# For each product time-series, check if the 'date' feature is unique
{product: df_product['date'].is_unique for product, df_product in df_product_dict.items()}

{'SKU1': True,
 'SKU2': True,
 'SKU3': True,
 'SKU4': True,
 'SKU5': True,
 'SKU6': True}

### Verify all data points have equal time frequency

In [21]:
# For each product time-series, get start date and end date
{product: (df_product['date'].min(),df_product['date'].max()) for product, df_product in df_product_dict.items()}

{'SKU1': (Timestamp('2017-02-05 00:00:00'), Timestamp('2020-12-27 00:00:00')),
 'SKU2': (Timestamp('2017-02-05 00:00:00'), Timestamp('2020-12-27 00:00:00')),
 'SKU3': (Timestamp('2017-02-05 00:00:00'), Timestamp('2020-12-27 00:00:00')),
 'SKU4': (Timestamp('2017-02-05 00:00:00'), Timestamp('2020-12-27 00:00:00')),
 'SKU5': (Timestamp('2017-02-05 00:00:00'), Timestamp('2020-12-27 00:00:00')),
 'SKU6': (Timestamp('2017-02-05 00:00:00'), Timestamp('2020-11-15 00:00:00'))}

In [22]:
# For each product time-series, count the number of date units (i.e., frequency)
{product: df_product['date'].nunique() for product, df_product in df_product_dict.items()}

{'SKU1': 204, 'SKU2': 204, 'SKU3': 204, 'SKU4': 204, 'SKU5': 204, 'SKU6': 198}

In [23]:
# Append missing date units to 'SKU6' time-series from 'SKU1' time-series
df_product_dict['SKU6'] = df_product_dict['SKU6'].merge(df_product_dict['SKU1'][['date']], how='outer', on=['date'])

In [24]:
# Fill null values for 'Product' feature with name 'SKU6'
df_product_dict['SKU6']['Product'] = df_product_dict['SKU6']['Product'].fillna('SKU6')

In [25]:
# For odd product 'SKU6' time-series, count the number of date units (i.e., frequency) after appending missing date units 
df_product_dict['SKU6']['date'].nunique()

204

In [26]:
# For odd product 'SKU6' time-series, get start date and end date after appending missing date units
(df_product_dict['SKU6']['date'].min(),df_product_dict['SKU6']['date'].max())

(Timestamp('2017-02-05 00:00:00'), Timestamp('2020-12-27 00:00:00'))

In [27]:
# Display time-series of odd product 'SKU6'
df_product_dict['SKU6']

,Product,date,Sales,Price Discount,In-Store Promo,Catalogue Promo,Store End Promo,Google_Mobility,Covid_Flag,V_DAY,EASTER,CHRISTMAS
0,SKU6,2017-02-05,32138.0,0.28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,SKU6,2017-02-12,11659.0,0.05,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,SKU6,2017-02-19,12140.0,0.05,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,SKU6,2017-02-26,29635.0,0.28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,SKU6,2017-03-05,11666.0,0.05,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
199,SKU6,2020-11-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200,SKU6,2020-12-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201,SKU6,2020-12-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202,SKU6,2020-12-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
date_unit = ((2020-2017)*12*31)/204

date_unit

5.470588235294118

In [29]:
# For each product time-series, get start date's day name and end date's day name
{product: (df_product['date'].min().day_name(),df_product['date'].max().day_name()) for product, df_product in df_product_dict.items()}

{'SKU1': ('Sunday', 'Sunday'),
 'SKU2': ('Sunday', 'Sunday'),
 'SKU3': ('Sunday', 'Sunday'),
 'SKU4': ('Sunday', 'Sunday'),
 'SKU5': ('Sunday', 'Sunday'),
 'SKU6': ('Sunday', 'Sunday')}

In [30]:
for product, df_product in df_product_dict.items():
    df_product = df_product.copy()
    # Set the 'date' column as index
    df_product.set_index('date', inplace=True)
    # Aggregate by week starting on Sunday
    df_product = df_product.resample('W-SUN').first()
    # Update df_product
    df_product_dict[product] = df_product

In [31]:
# Display time-series of one product
df_product_dict['SKU1']

,Product,Sales,Price Discount,In-Store Promo,Catalogue Promo,Store End Promo,Google_Mobility,Covid_Flag,V_DAY,EASTER,CHRISTMAS
date,,,,,,,,,,,
2017-02-05,SKU1,27750,0.00,0,0,0,0.00,0,0,0,0
2017-02-12,SKU1,29023,0.00,1,0,1,0.00,0,1,0,0
2017-02-19,SKU1,45630,0.17,0,0,0,0.00,0,0,0,0
2017-02-26,SKU1,26789,0.00,1,0,1,0.00,0,0,0,0
2017-03-05,SKU1,41999,0.17,0,0,0,0.00,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2020-11-29,SKU1,0,0.16,1,0,1,-5.30,1,0,0,0
2020-12-06,SKU1,0,0.15,1,0,1,-4.38,1,0,0,0
2020-12-13,SKU1,0,0.14,1,0,1,-4.74,1,0,0,0


In [32]:
# Display time-series of odd product
df_product_dict['SKU6']

,Product,Sales,Price Discount,In-Store Promo,Catalogue Promo,Store End Promo,Google_Mobility,Covid_Flag,V_DAY,EASTER,CHRISTMAS
date,,,,,,,,,,,
2017-02-05,SKU6,32138.0,0.28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-02-12,SKU6,11659.0,0.05,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2017-02-19,SKU6,12140.0,0.05,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2017-02-26,SKU6,29635.0,0.28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-03-05,SKU6,11666.0,0.05,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2020-11-29,SKU6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-06,SKU6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-13,SKU6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Save granular time-series into separate CSV files

In [33]:
for product, df_product in df_product_dict.items():
    df_product = df_product.copy()
    # Save df_product to a CSV file 
    df_product.to_csv(data_path+product.lower()+'.csv')